In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from urllib.parse import quote

데이터 불러오기 - 구석구석 데이터

In [ ]:
# path는 각자 로컬 환경에 맞게 변경해주세요
dataset_raw = pd.read_csv('../data/구석구석dataset_수정 (1).csv', encoding='cp949')
dataset = dataset_raw.copy()

In [ ]:
# 데이터 확인
dataset

,Unnamed: 0,attraction,location,tag
0,0,동대문역사문화공원,서울 중구,#2024_여행가는달과함께하는_발도장여행지#DDP#가족여행#걷기좋은길#관광지#데이트...
1,1,서울식물원,서울 강서구,#2024_여행가는달과함께하는_발도장여행지#가족과함께#관광지#교육여행#미세먼지_피하...
2,2,동대문디자인플라자(DDP),서울 중구,#19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...
3,3,N서울타워,서울 용산구,#19_20한국관광100선#2024_여행가는달과함께하는_발도장여행지#21_22한국관...
4,4,서울숲,서울 성동구,#2024_여행가는달과함께하는_발도장여행지#23_24한국관광100선#가족여행#걷기길...
...,...,...,...,...
3742,3742,이비스앰배서더 인사동,서울 종로구,#서울이비스앰배서더호텔#숙박#인사동숙박#인사동앰배서더#인사동호텔#종로구호텔
3743,3743,코스모스아트홀,서울 서초구,#공연장대여#그랜드피아노공연장#문화시설#서초구공연장#서초구아트홀#서초리사이틀장#코스...
3744,3744,번지피지오 옴니바디웍스청담,서울 강남구,#강남요가원#관광지#번지피지오#번지피지오옴니바디웍스청담#옴니바디웍스청담#취미운동#키...
3745,3745,커뮤니티하우스마실,서울 중구,#개러지홀#마실#명동마실#명동커뮤니티하우스마실#문화시설#페이지명동#프라이빗살롱


In [ ]:
# 관광지명 리스트 생성
dataset_values = dataset['attraction'].values

In [ ]:
# 관광지명 리스트 생성 확인
print(len(dataset_values))
print(dataset_values)

3747
['동대문역사문화공원' '서울식물원' '동대문디자인플라자(DDP)' ... '번지피지오 옴니바디웍스청담' '커뮤니티하우스마실'
 '씨에스에비뉴호텔(CS AVENUE호텔)']


In [ ]:
# 관광지명 리시트 인덱싱 확인
dataset_values[1600]

'호텔 삼정'

크롤링 코드 부분

- \# i번 으로 코드 진행 순서 명시해두었습니다. (코드 진행 흐름 파악용) >> 컨트롤 f 하셔서 i번 검색하시면 찾을 수 있습니다.
- a번 > b번 > c번 > d번 > e번 순
    - a번 : 구석구석 데이터 관광지명 리스트에서 관광지명 하나씩 추출해서 네이버 지도 창에 검색하는 코드
    - b번 : 검색된 창에서 검색 결과중 1) 관광지명과 가장 관련도가 큰 관광지 2) 주소가 서울특별시인 관광지 를 선택해서 들어가게하는 코드
    - c번 : 관광지 창에서 리뷰버튼 찾고 방문자 리뷰 '이런점이 좋았어요' 부분이 있으면 펼친 뒤 데이터 추출하는 함수 호출하는 코드
    - d번 : 데이터 추출하는 함수 코드
    - e번 : 추출된 데이터 DataFrame으로 만드는 코드

In [ ]:
# 웹 드라이버 설정
driver = webdriver.Chrome()
driver.maximize_window()

# 추출한 데이터 저장할 저장소 초기화
tourist_spot_list = []
good_point_list = []
selected_people_list = []
fail_spot_list = []
spot_attribute_list = []


# d번
# 데이터 추출 코드
def extract_data():
    try:
        tourist_spot = spot

        # 사용 가능한 '이런 점이 좋았어요' 요소 개수 파악
        # MHaAm : '이런 점이 좋았어요' 요소 CLASS NAME
        # 따라서 MHaAm 로 접근하여 각 요소 리스트에 담고 길이 구하기
        good_points_elements = driver.find_elements(By.CLASS_NAME, 'MHaAm')
        num_good_points = len(good_points_elements)

        # 첫번째 부터 마지막 요소번째 까지 반복
        for j in range(0, num_good_points + 1):
            try:
                # 좋은 점과 선택한 사람 수 추출
                good_point = driver.find_elements(By.CLASS_NAME, 't3JSf')[j].text # 이런 점이 좋았어요 - 이런점 추출
                selected_people = driver.find_elements(By.CLASS_NAME, 'CUoLy')[j].text # 이런 점이 좋았어요 - 투표 수 추출
                spot_attribute = driver.find_element(By.CLASS_NAME, 'lnJFt').text # 관광지 속성(네이버 지도 - 관광지명 옆 작은 카테고리명)

                # 추출된 요소들 리스트에 넣기 - 후반부 DataFrame 생성시 사용
                good_point_list.append(good_point)
                selected_people_list.append(selected_people)
                spot_attribute_list.append(spot_attribute)
                tourist_spot_list.append(tourist_spot)

            except Exception as e:
                print(f"항목 {j}은(는) 존재하지 않습니다. 오류: {e}")
                if 'driver' in locals():
                    driver.quit()
                return None

    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")

#--------------------------------------------------------------------------------------------------------------------
# 크롤링 코드 작성부

## a번
for j in range(0,15):

    # @창 띄우고 관광지 이름으로 검색하는 코드@
    # spot : 구석구석 관광지명 리스트에서 인덱싱된 관광지명
    spot = dataset_values[j]

    # 특수 문자 포함 관광지명 인코딩해주는 코드
    encoded_spot = quote(spot)

    # 지도 페이지로 바로 들어가도록 변경
    # 기존 코드와 다르게 접근
    driver.get(f"https://m.map.naver.com/search2/search.naver?query={encoded_spot}")
    time.sleep(5)

#---------------------------------------------------------------------------------------------------------------------
    # @지도창으로 들어와서 관광지 이름 버튼 클릭하는 코드@

    # b번
    # map_paths : 지도 검색 페이지에서 해당 관광지의 XPATH 모음 리스트
    map_paths = [
        '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/a/div/strong',
        '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/a[2]/div/strong'
    ]

    element_clicked = False
    timeout_count = 0 # TimeoutException 발생 횟수
    for map_path in map_paths:
        try:
            place_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, map_path))
            )

            # 주소 나오게 하는 버튼
            button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/div[1]/div/a/i'))
            )
            button.click()

            # 주소가 서울특별시로 시작하지 않는 경우 클릭되지 않게 만드는 코드
            adress = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/div[2]/div/p[1]').text
            adress_para = '서울특별시'

            if adress[0:5] == adress_para:
                place_button.click()
                element_clicked = True

        except TimeoutException:
            print(F"{spot}에 대한 관광지 버튼을 찾지 못했습니다.")
            timeout_count += 1 # 타임아웃 발생 시 횟수 1회씩 증가

    if not element_clicked:
        if timeout_count == 2:
            fail_spot_list.append(spot)
        continue

    # c번
    # @해당 관광지 창으로 들어와서 리뷰버튼 클릭하는 코드@
    review_xpaths = [
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[2]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[4]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[5]/span'
    ]

    try:
        review_found = False # 리뷰가 크롤링되면 나머지 review_xpath는 생략하도록 설정
        for review_xpath in review_xpaths:
            if review_found:
                break  # 리뷰를 찾았으면 루프를 종료

            review_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, review_xpath))
            )
            review_button.click()

            # 방문자 리뷰 - '이런점이 좋았어요' 펼치는 버튼 클릭 6번
            # button_class_name : 펼치는 버튼 CLASS NAME
            button_class_name = "dP0sq"

            for i in range(6):
                try:
                    first_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, button_class_name))
                    )
                    driver.execute_script("arguments[0].click();", first_button)
                    print(f"{spot} - {i+1}번째 버튼 클릭 성공")
                    # 다음 보기 버튼 누르는 것에 성공한다면 원하는 데이터 추출
                    # 함수 사용
                    extract_data()
                    review_found = True  # 리뷰를 찾았음을 표시 - 리뷰가 크롤링되면 나머지 review_xpath는 생략하도록 설정

                except TimeoutException:
                    print(f"{spot} - {i+1}번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.")
                    break

    except TimeoutException:
        print(F"{spot}에 대한 리뷰 버튼을 찾지 못했습니다.")
        continue

# e번
# 추출한 데이터 데이터 프레임 형식으로 만들기
data = {
    'tourist_spot': tourist_spot_list,
    'spot_arrtribute' : spot_attribute_list,
    'good_point': good_point_list,
    'selected_people': selected_people_list
}
df = pd.DataFrame(data)
df = df.drop_duplicates()

# 드라이버 종료
driver.quit()

새만포면옥 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
새만포면옥 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
새만포면옥 - 1번째 버튼 클릭 성공
항목 10은(는) 존재하지 않습니다. 오류: list index out of range
새만포면옥 - 2번째 버튼 클릭 성공
항목 15은(는) 존재하지 않습니다. 오류: list index out of range
새만포면옥 - 3번째 버튼 클릭 성공
항목 20은(는) 존재하지 않습니다. 오류: list index out of range
새만포면옥 - 4번째 버튼 클릭 성공
항목 25은(는) 존재하지 않습니다. 오류: list index out of range
새만포면옥 - 5번째 버튼 클릭 성공
항목 26은(는) 존재하지 않습니다. 오류: list index out of range
새만포면옥 - 6번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
플랫폼 플레이스(한남점) - 1번째 버튼 클릭 성공
항목 8은(는) 존재하지 않습니다. 오류: list index out of range
플랫폼 플레이스(한남점) - 2번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
브릿지안경 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
브릿지안경 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
브릿지안경 - 1번째 버튼 클릭 성공
항목 10은(는) 존재하지 않습니다. 오류: list index out of range
브릿지안경 - 2번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
한국금융사박물관 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
한국금융사박물관 - 1번째 버튼 클릭 성공
항목 7은(는) 존재하지 않습니다. 오류: list index out of range
한국금융사박물관 - 2번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
신당동 허리케인박 떡볶이 - 1번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
신당동 허리케인박 떡볶이에 대한 리

In [ ]:
# 생성된 데이터 프레임 확인
df

,tourist_spot,spot_arrtribute,good_point,selected_people
0,새만포면옥,냉면,"""음식이 맛있어요""",이 키워드를 선택한 인원\n517
1,새만포면옥,냉면,"""친절해요""",이 키워드를 선택한 인원\n230
2,새만포면옥,냉면,"""건강한 맛이에요""",이 키워드를 선택한 인원\n161
3,새만포면옥,냉면,"""재료가 신선해요""",이 키워드를 선택한 인원\n159
4,새만포면옥,냉면,"""매장이 청결해요""",이 키워드를 선택한 인원\n124
...,...,...,...,...
578,파로스관광호텔,3성급,"""인테리어가 멋져요""",이 키워드를 선택한 인원\n2
579,파로스관광호텔,3성급,"""주차하기 편해요""",이 키워드를 선택한 인원\n2
590,파로스관광호텔,3성급,"""사진이 잘 나와요""",이 키워드를 선택한 인원\n1
591,파로스관광호텔,3성급,"""수영장이 잘 되어있어요""",이 키워드를 선택한 인원\n1


In [ ]:
# 리뷰 크롤링 성공한 관광지 모음
df['tourist_spot'].unique()

array(['새만포면옥', '플랫폼 플레이스(한남점)', '브릿지안경', '한국금융사박물관', '황토추어탕', '해담채 삼성점',
       '까사빠보', '포포인츠 바이 쉐라톤 조선 서울역 더 이터리', '신촌 시스터즈', '오양회참치 광화문본점',
       'h450', '에프호텔', '파로스관광호텔'], dtype=object)

In [ ]:
# 리뷰 크롤링 성공한 관광지 개수 확인
len(df['tourist_spot'].unique())

13

In [ ]:
# 이름 변경이 필요한 관광지 모음
fail_spot

[]

In [ ]:
# 이름 변경이 필요한 관광지 개수 확인
len(fail_spot)

5

크롤링 결과는 3가지 유형이 존재
1. 성공 - 리뷰 '이런 점이 좋았어요' 가 존재하여 성공적으로 크롤링된 관광지
2. 실패 - 구석구석에 등록된 관광지명으로 네이버 지도 검색이 불가능해서 크롤링 실패된 관광지
3. 실패 - 구석구석에 등록된 관광지명으로 네이버 지도 검색이 가능하나 리뷰 크롤링이 불가능한 관광지
    - 3 유형은 크게 2가지 유형으로 나뉜다.
        1. 리뷰 '이런 점이 좋았어요' 가 존재하지 않아서 크롤링 되지 않는 경우 >> 문제없음 크롤링 안되는 게 맞음
        2. 리뷰 '이런 점이 좋았어요' 가 존재하지만 XPATH가 특이해서 크롤링 코드가 못가져 오는 경우 >> 다음 경우가 문제

- 따라서,
    - 2 유형 이름 확인 후 검색 되도록 수정 필요!!
    - 3-2 유형의 목록을 뽑고 수작업으로 확인 필요!!

In [ ]:
df.to_csv('../13_result.csv')